使用Pytorch中的卷积核进行卷积操作

In [1]:
import torch
import torch.nn as nn

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device.type)

cuda


In [3]:
# 定义一个卷积核，输入的通道数量为5，输出通道为2，卷积核大小（宽度）为4
conv1 = nn.Conv1d(5,2,4,device=device)
# 再定义一个卷积核，输入的通道数量为5，输出通道为2，卷积核大小（宽度）为3
conv2 = nn.Conv1d(5,2,3,device=device)

In [4]:
# 生成一个随机输出
inputs = torch.rand(2,5,6,device=device)
inputs.device

device(type='cuda', index=0)

In [5]:
output_1 = conv1(inputs)
output_1

tensor([[[-0.4302, -0.4778, -0.3028],
         [-0.0909, -0.1421,  0.0811]],

        [[-0.3165, -0.6822, -0.4207],
         [ 0.1178, -0.1478, -0.2259]]], device='cuda:0',
       grad_fn=<ConvolutionBackward0>)

In [6]:
output_2 = conv2(inputs)
output_2

tensor([[[-0.1495,  0.0107,  0.0274,  0.2551],
         [-0.5307, -0.4619, -0.1138, -0.2138]],

        [[ 0.2971,  0.1125,  0.2004, -0.1001],
         [-0.3739, -0.0624, -0.5236, -0.5240]]], device='cuda:0',
       grad_fn=<ConvolutionBackward0>)

使用Maxpool1d进行一维的最大池化操作

In [7]:
# 创建一个池化层，池化核大小为3
poo1 = nn.MaxPool1d(3)
# 创建一个池化层，池化核大小为4
pool2 = nn.MaxPool1d(4)
output_pool_1 = poo1(output_1)
output_pool_1

tensor([[[-0.3028],
         [ 0.0811]],

        [[-0.3165],
         [ 0.1178]]], device='cuda:0', grad_fn=<SqueezeBackward1>)

In [8]:
output_pool_2 = pool2(output_2)
output_pool_2

tensor([[[ 0.2551],
         [-0.1138]],

        [[ 0.2971],
         [-0.0624]]], device='cuda:0', grad_fn=<SqueezeBackward1>)

使用nn.functional.max_pool1d()函数直接调度池化函数

In [9]:
import torch.nn.functional as F
# 使用F.max_pool1d函数实现池化层，池化核大小为3（来自于output的第三层）
output_pool1 = F.max_pool1d(output_1,kernel_size=output_1.shape[2])
output_pool1

tensor([[[-0.3028],
         [ 0.0811]],

        [[-0.3165],
         [ 0.1178]]], device='cuda:0', grad_fn=<SqueezeBackward1>)

In [10]:
output_pool2 = F.max_pool1d(output_2,kernel_size=output_2.shape[2])
output_pool2

tensor([[[ 0.2551],
         [-0.1138]],

        [[ 0.2971],
         [-0.0624]]], device='cuda:0', grad_fn=<SqueezeBackward1>)

In [11]:
# 使用squeeze函数将output_pool1的第三层去掉，变成2*2的矩阵
# 因为output_pool1和output_pool2需要在后期进行拼接，所以需要将其变成相同的形状
output_pool1 = output_pool1.squeeze(dim=2)
output_pool2 = output_pool2.squeeze(dim=2)

In [13]:
output_pool1

tensor([[-0.3028,  0.0811],
        [-0.3165,  0.1178]], device='cuda:0', grad_fn=<SqueezeBackward1>)

In [14]:
output_pool2

tensor([[ 0.2551, -0.1138],
        [ 0.2971, -0.0624]], device='cuda:0', grad_fn=<SqueezeBackward1>)

In [16]:
outputs_pool = torch.cat((output_pool1,output_pool2),dim=1) # 指定按照第一维进行拼接
outputs_pool

tensor([[-0.3028,  0.0811,  0.2551, -0.1138],
        [-0.3165,  0.1178,  0.2971, -0.0624]], device='cuda:0',
       grad_fn=<CatBackward0>)

In [17]:
# 创建隐藏层转输出，输入为4，输出为2
linear = nn.Linear(4,2,device=device)
outputs_linear = linear(outputs_pool)

In [18]:
outputs_linear

tensor([[-0.3768,  0.4177],
        [-0.3858,  0.4158]], device='cuda:0', grad_fn=<AddmmBackward0>)